In [1]:
# -*- coding: utf-8 -*- 

import re 
from tqdm import tqdm 
import time
from datetime import datetime
import sqlite3
import sys
import os
import pandas as pd
import unify

In [2]:
def connect(file_path, primary, columns):
    con = sqlite3.connect(file_path)
    cur = con.cursor()
    cols = ", ".join([c + ' Varchar' for c in columns]) 
    cur.execute("create table meta ("+primary+" Varchar PRIMARY KEY, "+cols+" )")
    cur.execute("CREATE INDEX log on meta (textid);")
    cur.execute("create table plain_texts (id Varchar(128) NOT NULL PRIMARY KEY, text Varchar NOT NULL);")
    cur.execute("create table tagged_texts (id Varchar(128) NOT NULL PRIMARY KEY, text Varchar NOT NULL );")
    con.commit()
    return con, cur

In [11]:
workdir = r'/home/tari/Загрузки/taiga/Interfax'
filename = 'Interfax.db'
file_path = os.path.join(workdir, filename)
metatablepath = os.path.join(workdir,'newmetadata.csv')
tagged = os.path.join(workdir,'texts_tagged')
plain = os.path.join(workdir,'texts')

meta = pd.read_csv(metatablepath, sep='\t', encoding='utf8')
meta = meta.fillna('')
meta.head()

,segment,textid,textname,textregion,textrubric,textdiff,author,authortexts,authorreaders,magazine,date,time,tags,source
0,Interfax,world549261,На северо-востоке США объявлено штормовое пред...,,В мире,,,,,,2017-02-09,18:45,США,http://www.interfax.ru/world/549261
1,Interfax,russia549262,Путин выразил Эрдогану соболезнования после уд...,,В России,,,,,,2017-02-09,18:43,Сирия,http://www.interfax.ru/russia/549262
2,Interfax,world549259,В Германии суд отправил на доработку проект уг...,,В мире,,,,,,2017-02-09,18:33,Германия,http://www.interfax.ru/world/549259
3,Interfax,world549255,Трое турецких военных погибли в Сирии в резуль...,,В мире,,,,,,2017-02-09,18:15,Россия,http://www.interfax.ru/world/549255
4,Interfax,russia549254,В Крыму восстановили электроснабжение обесточе...,,В России,,,,,,2017-02-09,18:12,Крым,http://www.interfax.ru/russia/549254


In [12]:
meta.columns

Index(['segment', 'textid', 'textname', 'textregion', 'textrubric', 'textdiff',
       'author', 'authortexts', 'authorreaders', 'magazine', 'date', 'time',
       'tags', 'source'],
      dtype='object')

In [14]:
if not os.path.exists(filename):
    con, cur = connect(filename, meta.columns[1], [meta.columns[0]]+list(meta.columns[2:]))
else:
    con = sqlite3.connect(filename, meta.columns[1], [meta.columns[0]]+list(meta.columns[2:]))
    cur = con.cursor()

In [15]:
meta.iloc[6].to_dict()

{'author': '',
 'authorreaders': '',
 'authortexts': '',
 'date': '2017-02-09',
 'magazine': '',
 'segment': 'Interfax',
 'source': 'http://www.interfax.ru/world/549252',
 'tags': 'Баку',
 'textdiff': '',
 'textid': 'world549252',
 'textname': 'Российские дипломаты встретились в Баку с блогером Лапшиным',
 'textregion': '',
 'textrubric': 'В мире',
 'time': '18:06'}

In [ ]:
for i in tqdm(range(len(meta))):
    values = meta.iloc[i].to_dict()

    
    columns = ', '.join(values.keys())
    
    #print(list(values.values()))
    placeholders = ', '.join('?' * len(values))
    sql = 'INSERT INTO meta ({}) VALUES ({})'.format(columns, placeholders)
    #print(sql)
    cur.execute(sql, list(values.values()))
    
    valuest = {'id': values['textid'], 'text': unify.open_text(os.path.join(plain, str(values['textid'])+".txt"))}
    
    columns = ', '.join(valuest.keys())
    placeholders = ', '.join('?' * len(valuest))
    sql2 = 'INSERT INTO plain_texts ({}) VALUES ({})'.format(columns, placeholders) 
    cur.execute(sql2, list(valuest.values()))
    try:
        valuest2 = {'id': values['textid'], 'text': unify.open_text(open(os.path.join(tagged, str(values['textid'])+".txt"),'r', encoding='utf8').read())}
        columns = ', '.join(valuest2.keys())
        placeholders = ', '.join('?' * len(valuest2))
        sql3 = 'INSERT INTO tagged_texts ({}) VALUES ({})'.format(columns, placeholders) 
        cur.execute(sql3, list(valuest2.values()))
    except:
        valuest2 = {'id': values['textid'], 'text': ""}
        columns = ', '.join(valuest2.keys())
        placeholders = ', '.join('?' * len(valuest2))
        sql3 = 'INSERT INTO tagged_texts ({}) VALUES ({})'.format(columns, placeholders) 
        cur.execute(sql3, list(valuest2.values()))
    con.commit()